In [8]:
import numpy as np
import pandas as pd
import scipy.stats
from termcolor import colored

In [9]:
# Now, we can use pandas read_excel after installing the excel importer.

import pandas as pd

df = pd.read_excel('data.xlsx')
df

,Dilution Rate (1/h),Dry weight (g/L),Glucose Consumption Rate (C mol/C mol biomass),Oxygen Consumption Rate (mol O2/C mol biomass),Carbon Dioxide Production Rate (mol CO2/C mol biomass)
0,0.008,3.60,2.00,1.10,1.40
1,0.008,3.80,2.10,3.80,1.40
2,0.017,3.20,2.30,1.50,1.60
3,0.033,4.00,1.90,1.04,1.10
4,0.047,3.17,2.48,1.20,1.20
5,0.052,4.20,1.80,0.82,0.86
6,0.072,4.41,1.74,0.54,0.73
7,0.076,4.00,1.93,0.67,0.73
8,0.092,4.40,1.80,0.70,0.70
9,0.092,3.80,1.90,0.70,0.80


In [10]:
reactants = ['C6H12O6', 'O2', 'NH3']
products = ['CH1.83O0.56N0.17', 'H2O', 'CO2']

all_compounds = reactants + products
print(all_compounds)
error_percent_orig = [6, 11.7, None, 5, None, 11.1]
error_percent = [e for e in error_percent_orig if e is not None]

coeff_is_measured = [True, True, False, True, False, True]

['C6H12O6', 'O2', 'NH3', 'CH1.83O0.56N0.17', 'H2O', 'CO2']


In [11]:
col = df.columns
row = df.index

df2 = pd.DataFrame()

df2["D"] = df[col[0]]
df2["DW"] = df[col[1]]

df2['C6H12O6'] = -df[col[2]]/6
df2['O2'] = -df[col[3]]
df2['NH3'] = None
df2['CH1.83O0.56N0.17'] = 1.00
df2['H2O'] = None
df2['CO2'] = df[col[4]]
df2

,D,DW,C6H12O6,O2,NH3,CH1.83O0.56N0.17,H2O,CO2
0,0.008,3.60,-0.333333,-1.10,None,1.0,None,1.40
1,0.008,3.80,-0.350000,-3.80,None,1.0,None,1.40
2,0.017,3.20,-0.383333,-1.50,None,1.0,None,1.60
3,0.033,4.00,-0.316667,-1.04,None,1.0,None,1.10
4,0.047,3.17,-0.413333,-1.20,None,1.0,None,1.20
5,0.052,4.20,-0.300000,-0.82,None,1.0,None,0.86
6,0.072,4.41,-0.290000,-0.54,None,1.0,None,0.73
7,0.076,4.00,-0.321667,-0.67,None,1.0,None,0.73
8,0.092,4.40,-0.300000,-0.70,None,1.0,None,0.70
9,0.092,3.80,-0.316667,-0.70,None,1.0,None,0.80


In [12]:
def string_process1(compound):
    i = 0
    j = 1

    e_list = []

    while j < len(compound):
        if compound[j].isupper():
            e_list.append(compound[i:j])
            i = j
        j = j + 1
    if i != j:
        e_list.append(compound[i:j])

    element_map = dict()
    for e in e_list:

        for i in range(len(e)):
            if e[i].isdigit():
                key = e[0:i]
                value = float(e[i:len(e)])
                element_map[key] = value
                break
        else:
            element_map[e] = 1

    return element_map


# -----------------------------------------------
reactant_maps = []
product_maps = []

all_elements = set()  # all elements in the reaction for eg. C, H, Cl, K, N

for r in reactants:
    e_map = string_process1(r)  # element map for reactant r
    reactant_maps.append(e_map)
    all_elements = all_elements.union(e_map.keys())

for p in products:
    e_map = string_process1(p)  # element map for product p
    product_maps.append(e_map)
    all_elements = all_elements.union(e_map.keys())

all_elements = sorted(all_elements)
print("all element involved in chemical reaction are:", all_elements)

reactant_tuples = []
product_tuples = []

for r_map in reactant_maps:
    r_tuple = []
    for k in all_elements:
        if k in r_map:
            r_tuple.append(r_map[k])
        else:
            r_tuple.append(0)
    reactant_tuples.append(tuple(r_tuple))

for p_map in product_maps:
    p_tuple = []
    for k in all_elements:
        if k in p_map:
            p_tuple.append(p_map[k])
        else:
            p_tuple.append(0)
    product_tuples.append(tuple(p_tuple))

for i in range(len(reactants)):
    print("reactant_i =", reactants[i], ", -->", reactant_tuples[i])

for i in range(len(products)):
    print("product_i =", products[i], ", -->", product_tuples[i])

reactant_series = []
product_series = []

for i in range(len(reactants)):
    series_react_i = pd.Series(reactant_tuples[i], all_elements)
    reactant_series.append(-1 * series_react_i)

for i in range(len(products)):
    series_product_i = pd.Series(product_tuples[i], all_elements)
    product_series.append(series_product_i)

all_compounds_series = reactant_series + product_series
E_Orig = pd.DataFrame(dict(zip(all_compounds, all_compounds_series)))
E = E_Orig
display(E)
print()
A = np.array(E)
print("A =", '\n')
print(A, '\n')

all element involved in chemical reaction are: ['C', 'H', 'N', 'O']
reactant_i = C6H12O6 , --> (6.0, 12.0, 0, 6.0)
reactant_i = O2 , --> (0, 0, 0, 2.0)
reactant_i = NH3 , --> (0, 3.0, 1, 0)
product_i = CH1.83O0.56N0.17 , --> (1, 1.83, 0.17, 0.56)
product_i = H2O , --> (0, 2.0, 0, 1)
product_i = CO2 , --> (1, 0, 0, 2.0)


,C6H12O6,O2,NH3,CH1.83O0.56N0.17,H2O,CO2
C,-6.0,-0.0,-0.0,1.00,0.0,1.0
H,-12.0,-0.0,-3.0,1.83,2.0,0.0
N,-0.0,-0.0,-1.0,0.17,0.0,0.0
O,-6.0,-2.0,-0.0,0.56,1.0,2.0



A = 

[[ -6.    -0.    -0.     1.     0.     1.  ]
 [-12.    -0.    -3.     1.83   2.     0.  ]
 [ -0.    -0.    -1.     0.17   0.     0.  ]
 [ -6.    -2.    -0.     0.56   1.     2.  ]] 



In [13]:
def eliminate_unmeasured_coeff(E, all_coeff):
    element_balance_used = set()
    for i in coeff_not_measured_index:
        s = all_compounds_series[i]  # series for compound i

        x = (None, float('inf'))
        for e in all_elements:
            if (s[e] not in element_balance_used) and abs(s[e]) > 0 and abs(
                    s[e]) < x[1]:
                x = (e, abs(s[e]))
        # print("series =")
        # print(s, "\n")
        x = x[0]
        element_balance_used.add(x)
        #         print("We will use", x, "balance to derive the stiochiometric coefficient of " +
        #               all_compounds[i] + "\n")
        #         print("Performing row operations so that the stiochiometric coefficient of " + all_compounds[i] +
        #               " does not appear in other balances")

        other_elements = set(all_elements)
        other_elements.remove(x)
        # print("other elements =", other_elements)

        for e2 in other_elements:
            #             print("-------------- After doing row operation for ",
            #                   e2, "------------------", ' we get \n')
            #             print(" ")
            E.loc[e2] = E.loc[e2] - E.loc[x] * (s[e2] / s[x])
            #             display(E)
            #             print()
            A = np.array(E)


#             print("A =", '\n')
#             print(A, '\n')
#             print("*---------*-----------*----------*---------*---------*")

#     print("Since coefficients for", [all_compounds[i] for i in coeff_not_measured_index], "occurs, respectively, \
# in rows of ", element_balance_used, "only, \n We can drop rows", element_balance_used, " from matrix as well as corresponding\
# columns.\n These deleted elemental balances will be used to find values of unmeasured Stoichiometric coefficients\n")

#     print("--------deleting rows of ", element_balance_used, "we get ------------")
    E = E.drop(list(element_balance_used))
    display(E)
    #     print("*---------*-----------*----------*---------*---------*\n")

    #     print("--------deleting columns of ", [all_compounds[i] for i in coeff_not_measured_index], "we  get ------------")
    z = [all_compounds[i] for i in coeff_not_measured_index]
    E = E.drop(z, axis=1)
    display(E)
    #     print("*---------*-----------*----------*---------*---------*\n")
    return E


def perform_chi_square_test(E, x_measured, error_percent, confidence):
    #     display(E)
    print()
    A = np.array(E)
    #     print("A =", '\n')
    #     print(A, '\n')

    #     print("Checking the consistency of data")
    #     print("Ax =", '\n')
    #     print(A @ x_measured, '\n')

    error_abs = np.array(error_percent) * 0.01
    #     print("error_abs =", '\n')
    #     print(error_abs, '\n')

    N = len(error_percent)
    psi = np.zeros((N, N))
    #     print(psi)

    for i in range(N):
        psi[i, i] = (x_measured[i][0] * error_abs[i])**2


#     print("psi =", '\n')
#     print(psi, '\n')

    epsilon = -A @ x_measured
    #     print("epsilon =", '\n')
    #     print(epsilon, '\n')

    phi = A @ psi @ np.transpose(A)
    #     print("phi =", '\n')
    #     print(phi, '\n')

    # test function

    h = np.transpose(epsilon) @ np.linalg.inv(phi) @ epsilon
    h = h[0][0]

    #     print("the value of test function, h, is compared with chi-square distribution with m \
    # degrees of freedom \nwhere m = no of rows in our matrix A i.e. number of balances we have \
    # incorporated to find gross measurement errors\n")

    #     print("Since currently matrix A has dimension",
    #           np.shape(A), ", we have m = ", len(A))
    #     print("we will be using a confindence value of", confidence*100, "%.\n")

    #     print("h = ", end="")
    #     print(h, '\n')

    chi_square_value = scipy.stats.chi2.ppf(confidence, df=len(A))
    #     print("We have chi-square with", len(A), "degrees of freedom at",
    #           confidence*100, "% confidence level = ", chi_square_value, "\n")

    if h - chi_square_value <= 0:
        print(
            colored(
                "--------------------------------------------\n\
--------------Test Passed-------------------\n\
--------------------------------------------\n", "green"))
        return True, h
    else:
        print(
            colored(
                "********************************************\n\
*************Test Failed********************\n\
********************************************\n", "red"))
        return False, h


def perform_serial_elimination(matrix_E, confidence):
    #     print("Performing Serial Elimination Over matrix E shown just below\n")
    # display(matrix_E)
    #     print("we will we deleting one coefficient at a time and then again repeat the statistical analysis\n")
    element_balance_left = list(matrix_E.index)

    test_passed = []
    h_values_after_deletion = {}
    for i in range(len(all_coeff)):
        if all_coeff[i] is not None:
            E = matrix_E.copy()
            print("deleting the measurement for", all_compounds[i])
            print(
                "performing row operations and delting corresponding row and column\n"
            )
            s = E[all_compounds[i]]  ## series for compound i

            x = (None, float('inf'))
            for e in element_balance_left:
                if abs(s[e]) > 0 and abs(s[e]) < x[1]:
                    x = (e, abs(s[e]))
            # print("series =")
            # print(s, "\n")
            x = x[0]

            # print("We will use", x, "balance to derive the stiochiometric coefficient of " + all_compounds[i] + "\n")
            other_elements = set(element_balance_left)
            other_elements.remove(x)
            # print("other elements =", other_elements)
            # print("Performing row operations so that the stiochiometric coefficient of " + all_compounds[i] + " does not appear in other balances\n")

            for e2 in other_elements:
                # print("-------------- After doing row operation for ", e2, "------------------", ' we get \n')
                # print(" ")
                E.loc[e2] = E.loc[e2] - E.loc[x] * (s[e2] / s[x])
                # display(E)
                # print()
                # A = np.array(E)
                # print("A =", '\n')
                # print(A, '\n')
                # print("*---------*-----------*----------*---------*---------*")

                #                 print("Since coefficients for", all_compounds[i], "occur in row of ", x, "only, \n We can drop row for ", x, \
                # " from matrix as well as corresponding columns.\n")
                # print("--------deleting rows of ", x, "we get ------------")
                E = E.drop([x])
                # display(E)

                # print("--------deleting columns of ", all_compounds[i], "we  get ------------")
                z = [all_compounds[i]]
                E = E.drop(z, axis=1)
                # display(E)

                # print("*---------*-----------*----------*---------*---------*\n")

            # display(E)
            # print("*---------*-----------*----------*---------*---------*\n")


#             print("we will now repeat the same procedure i.e. compute the value of test function\n\
# and compare it chi-square distribution value\n")
            coeff_left = []
            error_percent2 = []
            for j in range(len(all_coeff)):
                if j != i and all_coeff[j] is not None:
                    coeff_left.append(all_coeff[j])
                    error_percent2.append(error_percent_orig[j])

            x_measured2 = np.array([[c] for c in coeff_left])
            # print("x_measured ----> ", x_measured)

            result, h = perform_chi_square_test(E, x_measured2, error_percent2, confidence)
            h_values_after_deletion[all_compounds[i]] = h;
            if result:
                test_passed.append(i)
    print(
        colored(
            "\nDeletion of stoichiometric coefficients of " +
            str([all_compounds[i]
                 for i in test_passed]) + " passes the chi-square test",
            'blue'))
    print(
        colored(
            "Hence " + str([all_compounds[i] for i in test_passed]) +
            " measurements can be suspected of containg GROSS ERRORS", 'blue'))
    return h_values_after_deletion;

In [14]:
h_no_deletion = []
h_val = {}

for i in range(len(all_compounds)):
    if coeff_is_measured[i]:
        h_val[all_compounds[i]] = []
        

for i in df.index:
    all_coeff = []
    for z in df2.loc[i].values[2:]:
        if z is not None:
            z = round(abs(z), 4)
        all_coeff.append(z)
    print("i = ", i)
    coeff_not_measured_index = []

    for j in range(len(all_coeff)):
        if all_coeff[j] == None:
            coeff_not_measured_index.append(j)

    print("all_coeff = ", all_coeff)
    print("coeff not measured", coeff_not_measured_index)

    E1 = E_Orig.copy()
    E = eliminate_unmeasured_coeff(E1, all_coeff)
    A = np.array(E)
    x_measured = np.array([[i] for i in all_coeff if i is not None])
    print("x_measured = ", x_measured)
    confidence = 0.9
    test_passed, h_orig = perform_chi_square_test(E, x_measured,
                                                  error_percent, confidence)
    print("h_org =", h_orig)
    h_no_deletion.append(h_orig);
    
    # if test_passed:
    #     print("since h <= chi-square with", len(A), "degrees of freedom at",
    #         confidence * 100, "% confidence level, \n\
    # we can say that gross errors are not present in system.")
    # else:
    #     print("since h >> chi-square with", len(A), "degrees of freedom at",
    #         confidence * 100, "% confidence level, \n\
    # we can state with 90% of confidence that some of the measurements have gross error.")
    #     print("Hence we proceed with serial elimination algorithm to find the location of possible gross errors.\n")

    h_values_serial_elimination = perform_serial_elimination(E, confidence)

    print("-----------------------------")
    display(h_values_serial_elimination)
    print("-------------------------------")
    for k in h_values_serial_elimination:
        h_val[k].append(h_values_serial_elimination[k])

i =  0
all_coeff =  [0.3333, 1.1, None, 1.0, None, 1.4]
coeff not measured [2, 4]


,C6H12O6,O2,NH3,CH1.83O0.56N0.17,H2O,CO2
C,-6.0,0.0,0.0,1.0,0.0,1.0
H,0.0,4.0,0.0,0.2,0.0,-4.0


,C6H12O6,O2,CH1.83O0.56N0.17,CO2
C,-6.0,0.0,1.0,1.0
H,0.0,4.0,0.2,-4.0


x_measured =  [[0.3333]
 [1.1   ]
 [1.    ]
 [1.4   ]]

--------------------------------------------
--------------Test Passed-------------------
--------------------------------------------

h_org = 3.9112613225134294
deleting the measurement for C6H12O6
performing row operations and delting corresponding row and column


--------------------------------------------
--------------Test Passed-------------------
--------------------------------------------

deleting the measurement for O2
performing row operations and delting corresponding row and column


********************************************
*************Test Failed********************
********************************************

deleting the measurement for CH1.83O0.56N0.17
performing row operations and delting corresponding row and column


--------------------------------------------
--------------Test Passed-------------------
--------------------------------------------

deleting the measurement for CO2
performing row ope

{'C6H12O6': 1.534906223369375,
 'O2': 3.901937993847941,
 'CH1.83O0.56N0.17': 1.6866828142528685,
 'CO2': 0.6690986670738737}

...

,C6H12O6,O2,NH3,CH1.83O0.56N0.17,H2O,CO2
C,-6.0,0.0,0.0,1.0,0.0,1.0
H,0.0,4.0,0.0,0.2,0.0,-4.0


,C6H12O6,O2,CH1.83O0.56N0.17,CO2
C,-6.0,0.0,1.0,1.0
H,0.0,4.0,0.2,-4.0


x_measured =  [[0.285]
 [0.57 ]
 [1.   ]
 [0.67 ]]

--------------------------------------------
--------------Test Passed-------------------
--------------------------------------------

h_org = 0.5381803569831478
deleting the measurement for C6H12O6
performing row operations and delting corresponding row and column


--------------------------------------------
--------------Test Passed-------------------
--------------------------------------------

deleting the measurement for O2
performing row operations and delting corresponding row and column


--------------------------------------------
--------------Test Passed-------------------
--------------------------------------------

deleting the measurement for CH1.83O0.56N0.17
performing row operations and delting corresponding row and column


--------------------------------------------
--------------Test Passed-------------------
--------------------------------------------

deleting the measurement for CO2
performing row operati

{'C6H12O6': 0.2503830108917622,
 'O2': 0.08621778108205046,
 'CH1.83O0.56N0.17': 0.21794062163962588,
 'CO2': 0.4568382055212549}

-------------------------------


In [15]:


analysis = pd.DataFrame()
analysis["D"] = df[col[0]]
analysis["DW"] = df[col[1]]

analysis['h (no deletion)'] = np.round(h_no_deletion, 2)


for i in range(len(all_compounds)):
    if coeff_is_measured[i]:
        analysis["h after deleting " + str(all_compounds[i])] = np.round(h_val[all_compounds[i]], 2)

display(analysis)    

,D,DW,h (no deletion),h after deleting C6H12O6,h after deleting O2,h after deleting CH1.83O0.56N0.17,h after deleting CO2
0,0.008,3.60,3.91,1.53,3.90,1.69,0.67
1,0.008,3.80,35.07,27.06,2.12,26.43,34.96
2,0.017,3.20,2.07,0.04,1.70,0.06,1.19
3,0.033,4.00,1.65,0.00,1.31,0.01,1.18
4,0.047,3.17,1.99,0.07,1.85,0.10,1.18
5,0.052,4.20,0.23,0.01,0.15,0.00,0.21
6,0.072,4.41,2.42,1.86,0.01,1.73,1.27
7,0.076,4.00,2.20,0.01,1.78,0.00,1.98
8,0.092,4.40,0.53,0.20,0.50,0.23,0.12
9,0.092,3.80,1.01,0.17,0.43,0.13,1.00


In [32]:
df3 = df2.copy()
df3['RQ'] = df3['CO2'] / abs(df3['O2'])
df3

,D,DW,C6H12O6,O2,NH3,CH1.83O0.56N0.17,H2O,CO2,RQ
0,0.008,3.60,-0.333333,-1.10,None,1.0,None,1.40,1.272727
1,0.008,3.80,-0.350000,-3.80,None,1.0,None,1.40,0.368421
2,0.017,3.20,-0.383333,-1.50,None,1.0,None,1.60,1.066667
3,0.033,4.00,-0.316667,-1.04,None,1.0,None,1.10,1.057692
4,0.047,3.17,-0.413333,-1.20,None,1.0,None,1.20,1.000000
5,0.052,4.20,-0.300000,-0.82,None,1.0,None,0.86,1.048780
6,0.072,4.41,-0.290000,-0.54,None,1.0,None,0.73,1.351852
7,0.076,4.00,-0.321667,-0.67,None,1.0,None,0.73,1.089552
8,0.092,4.40,-0.300000,-0.70,None,1.0,None,0.70,1.000000
9,0.092,3.80,-0.316667,-0.70,None,1.0,None,0.80,1.142857
